<table width="100%" border="3">
  <tbody>
    <tr>
      <td ><img src="https://aidea-web.tw/images/web/logo_white.png" alt="Aidea" width="400"/></td>
      <td align='left'><h1>AUAI Exercise 2: AOI Simple Pipeline </h1></td>
    </tr>
  </tbody>
</table>

# Preparation of TensorFlow and AOI Dataset

## Step 1: Import TensorFlow & other libraries

In [ ]:
import tensorflow as tf
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Step 2:Download AOI dataset to Colab from Google drive

https://drive.google.com/file/d/1Arcy_6pkMwLrg05A4f8S5dRzyu4I5j8Q/view?usp=sharing

In [ ]:
from google_drive_downloader import GoogleDriveDownloader
GoogleDriveDownloader.download_file_from_google_drive(file_id='1Arcy_6pkMwLrg05A4f8S5dRzyu4I5j8Q',dest_path='./content', unzip=True)

## Step 3:Check input data 

In [ ]:
import pandas as pd
df_train = pd.read_csv("train.csv")
print(df_train.shape)

In [ ]:
df_train.head()

In [ ]:
train_num = df_train.shape[0]
train_files = df_train.iloc[:train_num,0].values
train_labels = df_train.iloc[:train_num,1].values
print(train_labels[:20])

In [ ]:
import seaborn as sns
g = sns.countplot(train_labels)

# IPO-M: Model

<img src="https://miro.medium.com/max/1571/1*XB4SlSGxGKFQbIBoil0aDg.png" alt="Pre-train models" width="500">

## Step 4:Load ImageNet CNN Model

In [ ]:
num_classes=6

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dropout, Flatten, Activation
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop

In [ ]:
from tensorflow.keras.applications import InceptionResNetV2
model = InceptionResNetV2(include_top = True, input_shape=(299,299,3), weights=None, classes=num_classes)

In [ ]:
model.summary()

## Step 5: Compile model

In [ ]:
#compile model using accuracy to measure model performance
#model.summary()
from tensorflow.keras import optimizers
model.compile(loss='categorical_crossentropy',
            optimizer=optimizers.Adam(lr=3e-3),
            metrics=['accuracy'])

# IPO-I: Encode input images

## Step 6: read train images

In [ ]:
train_path = "train_images/"
train_images = []
from tensorflow.keras.preprocessing import image
for file in train_files:
    img = image.load_img(train_path+file, color_mode="rgb", target_size = (299, 299))
    train_images.append(img)
    if len(train_images)%100 == 0:
      print('.', end='')
print(len(train_images))

## Step 7: convert train images into np.array

In [ ]:
arr = []
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.inception_v3 import preprocess_input
for img in train_images:
    x = image.img_to_array(img)
    img_array = preprocess_input(x)
    arr.append(img_array)
X_train = np.array(arr)
print(X_train.shape)

# IPO-O: Encode output labes

## Step 8: convert train label into  onehot array

In [ ]:
from tensorflow.keras.utils import to_categorical
# one-hot encoding
num_classes = 6
y_train = to_categorical(train_labels, num_classes)
print(y_train)

## IPO-P: Model training

## Step 9: train our model 

In [ ]:
#train the model
hist = model.fit(X_train, y_train, batch_size=20, epochs=10)

## Step 10: save our model 

In [ ]:
model.save("AOI-inceptionv3-0812.h5")

# IPO-P: Model evaluation

## Step 11: Show training loss

In [ ]:
plt.plot(hist.history['loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train'])
plt.show()

## Step 11: Show training accuracy

In [ ]:
plt.plot(hist.history['accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train'])
plt.show()

## Step 11: check the prediction for training set

In [ ]:
y_prediction = model.predict(X_train, batch_size=20)
print(y_prediction[:2])

In [ ]:
predict = np.argmax(y_prediction,axis=1)
print(predict[0:10])

In [ ]:
print(train_labels[:10])

## Step 12: Compute confusion matrix (混淆矩陣)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion=confusion_matrix(train_labels, predict)
print(confusion)

## Step 13: Plot the confusion matrix

In [ ]:
import seaborn as sn
df_cm = pd.DataFrame(confusion)
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True, cmap="Blues")

## Step 14: List overkills and underkills

In [ ]:
overkill= []
underkill = []
for i in range(train_num):
  if train_labels[i] == 0 and predict[i] !=0:
    overkill.append(i)
  if train_labels[i] != 0 and predict[i] ==0:
    underkill.append(i)
print('# of overkill= {}; # of underkill= {} '.format(len(overkill), len(underkill)))

## Step 15: Check overkills

In [ ]:
#overkill
overkill_num = len(overkill)
no = 0
fig,ax=plt.subplots(2, 3)
fig.set_size_inches(10,10)
for i in range(2):
    for j in range (3):
        if no>=overkill_num:
          break 
        sel = overkill[no]
        ax[i,j].imshow(train_images[sel], cmap='gray')
        ax[i,j].set_title('No. {} Predicted Label:{} '.format(sel, predict[sel])) 
        no += 1
plt.tight_layout()

## Step 16: Check underkills

In [ ]:
#underkill
underkill_num = len(underkill)
no = 0
fig,ax=plt.subplots(2, 3)
fig.set_size_inches(10,10)
for i in range(2):
    for j in range (3):
        if no>=underkill_num:
          break 
        sel = underkill[no]
        ax[i,j].imshow(train_images[sel], cmap='gray')
        ax[i,j].set_title('No. {} train_labels:{} '.format(sel, train_labels[sel])) 
        no += 1
plt.tight_layout()

## Step 17: Delete training data in memory

In [ ]:
del train_images
del X_train
#Do GC
import gc
gc.collect()

# Method 1 of input test data

# IPO-I: input test data

## Step 18:Check test data 

In [ ]:
df_test = pd.read_csv("test.csv")
print(df_test.shape)

In [ ]:
df_test.head()

## Step 19: Limit the number of test images for input of CNN model

In [ ]:
#test_num = df_test.shape[0]
test_num = 2000 #limit the amount of training images for the class process
if test_num >= df_test.shape[0]:
  test_num = df_test.shape[0]

In [ ]:
test_files  = df_test.iloc[:test_num,0].values
test_labels = df_test.iloc[:test_num,1].values
print(test_labels[:10])

## Step 20: read test images

Note the limited number of test images for input of CNN model

In [ ]:
len(test_files)

In [ ]:
test_path = "test_images/"
test_images = []
for file in test_files:
    img = image.load_img(test_path+file, color_mode="rgb", target_size = (299, 299))
    test_images.append(img)
    if len(test_images)%100 == 0:
      print('.', end='')
print(len(test_images))

## Step 21: encode test images into np.array

In [ ]:
arr = []
for img in test_images:
    x = image.img_to_array(img)
    img_array = preprocess_input(x)
    arr.append(img_array)
X_test = np.array(arr)
print(X_test.shape)

# IPO-O: output predictions for test images by our model

## Step 22: model.predict() for test images 

In [ ]:
y_prediction = model.predict(X_test, batch_size=20)
predict = np.argmax(y_prediction,axis=1)
print(predict[:20])

## Step 23: write csv file for 

In [ ]:
df_out = pd.DataFrame(predict)
df_out.shape

In [ ]:
df_out.to_csv("result-20210812.csv", index=False)

# Method 2 of input test data

# Step 31: Load the test set 

In [ ]:
df_test = pd.read_csv("test.csv",dtype=str)
print(df_test.shape)

In [ ]:
df_test.head()

In [ ]:
test_files  = df_test.iloc[:,0].values
test_labels = df_test.iloc[:,1].values
print(test_labels[:10])

## Step 32: Instancing an ImageDataGenerator

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
img_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

## Step 33: Set up a test_generator with flow_from_dataframe

In [ ]:
test_generator = img_gen.flow_from_dataframe(dataframe=df_test,
            directory="test_images",
            x_col="ID",
            y_col="Label",
            batch_size=32,
            shuffle=False,
            class_mode=None,
            target_size=(299,299))

## Step 34: step_size_test

In [ ]:
if test_generator.n % test_generator.batch_size ==0:
  step_size_test=test_generator.n//test_generator.batch_size
else:
  step_size_test=test_generator.n//test_generator.batch_size + 1
print(step_size_test)

In [ ]:
#y_predictions = model.predict(X_train, batch_size=20)
test_generator.reset()
y_predictions = model.predict_generator(generator=test_generator, steps=step_size_test,verbose=1)

In [ ]:
import numpy as np
predicts=np.argmax(y_predictions,axis=1)
predicts[:10]

## Step 35: Output test results

In [ ]:
df_out = pd.DataFrame(df_test)
df_out.shape

In [ ]:
df_out['Label'] = predicts
df_out.to_csv("submit-20210812.csv", index=False)